In [0]:
import numpy as np

In [0]:
arr = np.arange(32)

In [0]:
v = arr.reshape([4,2,4])

In [17]:
v

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]],

       [[24, 25, 26, 27],
        [28, 29, 30, 31]]])

In [18]:
print(v.sum(axis=0), v.sum(axis=0).shape) # 행

[[48 52 56 60]
 [64 68 72 76]] (2, 4)


In [19]:
print(v.sum(axis=1), v.sum(axis=1).shape) # 열

[[ 4  6  8 10]
 [20 22 24 26]
 [36 38 40 42]
 [52 54 56 58]] (4, 4)


In [20]:
print(v.sum(axis=2), v.sum(axis=2).shape) # 깊이

[[  6  22]
 [ 38  54]
 [ 70  86]
 [102 118]] (4, 2)


In [21]:
mytensor = np.arange(2*3*4).reshape(2,3,4)
mytensor

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [22]:
mytensor.shape

(2, 3, 4)

In [23]:
np.transpose(mytensor).shape

(4, 3, 2)

In [24]:
np.transpose(mytensor, [1,2,0]).shape # '현재' 0,1,2가 저 모양으로 이동

(3, 4, 2)

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [27]:
mnist = input_data.read_data_sets("./mnist2/data.", one_hot=True)

W0821 05:35:48.094730 139935551059840 deprecation.py:323] From <ipython-input-27-ce16def223e5>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0821 05:35:48.096376 139935551059840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0821 05:35:48.098838 139935551059840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be remove

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist2/data./train-images-idx3-ubyte.gz


W0821 05:35:48.733147 139935551059840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0821 05:35:48.738938 139935551059840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist2/data./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist2/data./t10k-images-idx3-ubyte.gz


W0821 05:35:49.012748 139935551059840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist2/data./t10k-labels-idx1-ubyte.gz


In [0]:
learning_rate = 0.001
total_epoch = 30
batch_size = 128
tf.reset_default_graph()

In [0]:
n_input = 28 # 가로 픽셀수
n_step = 28 # 세로 픽셀수(입력 단계, step)
n_hidden = 128
n_class = 10
tf.reset_default_graph()

In [0]:
x = tf.placeholder(tf.float32, [None,n_step,n_input])
y = tf.placeholder(tf.float32, [None,n_class])
w = tf.get_variable("w_1", [n_hidden, n_class], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable("b_1", [n_class], initializer=tf.contrib.layers.xavier_initializer())

In [0]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
# Y : [batch_size, n_class]
# outputs : [batch_size, n_step, n_hidden]
# => [n_step, batch_size, n_hidden]
# => [batch_size, n_hidden] # 모양을 바꿔서 사용해야 함!
outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]  # 일케하면 [batch_size, n_hidden]만 남음!
model = tf.matmul(outputs,w) + b

cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [54]:
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
  total_cost = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
    # [batch_size, n_step, n_input]
    _, cv = sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys})
    total_cost += cv
  print("epoch : {}, avg_cost : {}".format(epoch+1, total_cost/total_batch))

epoch : 1, avg_cost : 0.5430568915147048
epoch : 2, avg_cost : 0.2515267971512321
epoch : 3, avg_cost : 0.1851929910741486
epoch : 4, avg_cost : 0.15459033640184047
epoch : 5, avg_cost : 0.1326642122275971
epoch : 6, avg_cost : 0.11775518754076013
epoch : 7, avg_cost : 0.11380404850906564
epoch : 8, avg_cost : 0.1036257049186038
epoch : 9, avg_cost : 0.0970652755697221
epoch : 10, avg_cost : 0.09360932667103611
epoch : 11, avg_cost : 0.08757326683129395
epoch : 12, avg_cost : 0.08845197211186855
epoch : 13, avg_cost : 0.07671782077835017
epoch : 14, avg_cost : 0.07741638352673301
epoch : 15, avg_cost : 0.07457666702585898
epoch : 16, avg_cost : 0.07290003528180239
epoch : 17, avg_cost : 0.07098801025256758
epoch : 18, avg_cost : 0.06399800484274928
epoch : 19, avg_cost : 0.06971881681692836
epoch : 20, avg_cost : 0.06592006385565186
epoch : 21, avg_cost : 0.06018876325716818
epoch : 22, avg_cost : 0.06557242143551856
epoch : 23, avg_cost : 0.05725143788955533
epoch : 24, avg_cost : 0.0

In [56]:
is_correct = tf.cast(tf.equal(tf.argmax(model,1), tf.argmax(y,1)), dtype=tf.float32)
accuracy = tf.reduce_mean(is_correct)
test_batch_size = len(mnist.test.images)

test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print("정확도 : ", sess.run(accuracy, feed_dict={x:test_xs, y:test_ys}) )

정확도 :  0.9739


In [0]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

In [58]:
num_dic = {n:i for i,n in enumerate(char_arr)}
num_dic

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [139]:
word_dic = {i:n for i,n in enumerate(char_arr)}
word_dic

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z'}

In [66]:
from keras.utils.np_utils import to_categorical

word_list = ['test','free','make','show','take','four']

to_categorical(num_dic[word_list[0][0]], 26)

t
e
x
t


In [0]:
ohe_list = []

for word in word_list:
  tmp = []
  for ch in word:
    tmp.append(to_categorical(num_dic[ch], 26))
  ohe_list.append(tmp)

In [80]:
np.shape(ohe_list)

(6, 4, 26)

In [0]:
word_input = np.array(ohe_list)

In [92]:
word_input.shape

(6, 4, 26)

In [0]:
word_x = word_input[:,:-1,:]

In [94]:
word_x.shape

(6, 3, 26)

In [0]:
word_y = word_input[:,-1,:]

In [0]:
word_y = word_y.reshape(6,26)

In [109]:
word_y.shape

(6, 26)

In [0]:
learning_rate = 0.01
total_epoch = 30
batch_size = 2

In [0]:
n_input = 26 # 가로 픽셀수
n_step = 3 # 세로 픽셀수(입력 단계, step)
n_hidden = 128
n_class = 26
tf.reset_default_graph()

In [0]:
x = tf.placeholder(tf.float32, [None,n_step,n_input])
y = tf.placeholder(tf.float32, [None,n_class])
w = tf.get_variable("w_1", [n_hidden, n_class], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable("b_1", [n_class], initializer=tf.contrib.layers.xavier_initializer())

In [0]:
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
# Y : [batch_size, n_class]
# outputs : [batch_size, n_step, n_hidden]
# => [n_step, batch_size, n_hidden]
# => [batch_size, n_hidden] # 모양을 바꿔서 사용해야 함!
outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]  # 일케하면 [batch_size, n_hidden]만 남음!
model = tf.matmul(outputs,w) + b

cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
import pandas as pd

In [0]:
predict = tf.argmax(model,1)
real = tf.argmax(y,1)
mypred = sess.run(predict, feed_dict={x:word_x})
real_y = sess.run(real, feed_dict={y:word_y})

In [153]:
for epoch in range(total_epoch):
  _, cv, mp, ry = sess.run([optimizer, cost, predict, real], feed_dict={x:word_x, y:word_y})
  print("epoch : {}, cost : {}".format(epoch+1, cv))
  for p, r in zip(mp, ry):
    print("pred: ", word_dic[p], "real: ", word_dic[r])

epoch : 1, cost : 3.1727454662323
pred:  j real:  t
pred:  t real:  e
pred:  t real:  e
pred:  t real:  w
pred:  t real:  e
pred:  t real:  r
epoch : 2, cost : 2.9977004528045654
pred:  t real:  t
pred:  t real:  e
pred:  t real:  e
pred:  t real:  w
pred:  t real:  e
pred:  t real:  r
epoch : 3, cost : 2.7870256900787354
pred:  t real:  t
pred:  e real:  e
pred:  e real:  e
pred:  t real:  w
pred:  e real:  e
pred:  t real:  r
epoch : 4, cost : 2.4853992462158203
pred:  t real:  t
pred:  e real:  e
pred:  e real:  e
pred:  w real:  w
pred:  e real:  e
pred:  e real:  r
epoch : 5, cost : 2.0406529903411865
pred:  t real:  t
pred:  e real:  e
pred:  e real:  e
pred:  w real:  w
pred:  e real:  e
pred:  e real:  r
epoch : 6, cost : 1.4556695222854614
pred:  e real:  t
pred:  e real:  e
pred:  e real:  e
pred:  w real:  w
pred:  e real:  e
pred:  e real:  r
epoch : 7, cost : 0.9532667994499207
pred:  e real:  t
pred:  e real:  e
pred:  e real:  e
pred:  e real:  w
pred:  e real:  e
pred: 

In [0]:
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(y, 1))
accuracy = tf.cast(is_correct, dtype=tf.float32)

In [0]:
predict = tf.argmax(model,1)
real = tf.argmax(y,1)
mypred = sess.run(predict, feed_dict={x:word_x})
real_y = sess.run(real, feed_dict={y:word_y})

In [140]:
for p, r in zip(mypred, real_y):
  print("pred: ", word_dic[p], "real: ", word_dic[r])

pred:  t real:  t
pred:  e real:  e
pred:  e real:  e
pred:  w real:  w
pred:  e real:  e
pred:  r real:  r


In [134]:
real_y

array([19,  4,  4, 22,  4, 17])